In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
file_dir = '/Users/markrademaker/Projects/Spatio_temporal_clustering/Data/'

## Separation in Juvenile and Adult Biomass

In [ ]:
def make_identifier(df):
    str_id = df.apply(lambda x: '_'.join(map(str, x)), axis=1)
    return pd.factorize(str_id)[0]

#List of species included in research
species_list = ["Limanda_limanda","Merlangius_merlangus","Clupea_harengus","Gadus_morhua","Sprattus_sprattus",
                "Pleuronectes_platessa","Eutrigla_gurnardus","Amblyraja_radiata","Callionymus_lyra"]

#life stages to separate
life_stage=["juvenile","adult"]

# list of tuples containing size in cm at maturation to separate juveniles from adults (based on FishBase)
length_class_list = [(11),  # Limanda limanda
                     (20.2),# Merlangius merlangus
                     (12.5),# Clupea harengus
                     (31),  # Gadus morhua
                     (8),   # Sprattus sprattus
                     (24),  # Pleuronectes platessa
                     (14.5),# Eutrigla gurnardus
                     (40),  # Amblyraja radiata
                     (17.4)]# Callionymus lyra


length_to_weight = [(0.0068,3.14), #parameter a,b  a*length^b conversion for Limanda limanda
                   (0.0063,3.06),  # ... Merlangius merlangus
                   (0.0060,3.08),  # ... Clupea harengus
                   (0.0071,3.08),  # ... Gadus morhua
                   (0.0056,3.09),  # ... Sprattus sprattus
                   (0.0089,3.04),  # ... Pleuronectes platessa
                   (0.0079,3.02),  # ... Eutrigla gurnardus
                   (0.0105,2.94),  # ... Amblyraja radiata
                   (0.0214,2.59)]  # ... Callionymus lyra

#North Sea coordinate extent:
min_lat= 48.00
max_lat= 62.00
min_lon= -5.00
max_lon= 13.01

#process per species
for i in range(len(species_list)):   
    species = species_list[i]
    print(species)
        
    df1=pd.read_csv(file_dir+species_list[i]+'.csv')

    #drop observations outside geographic range
    df1=df1.drop(df1[(df1["ShootLong"] < min_lon) & (df1["ShootLong"] > max_lon)].index)
    df1=df1.drop(df1[(df1["ShootLat"] < min_lat) & (df1["ShootLat"] > max_lat)].index)
    
    #Keep observations of quarter 1 (consistent 1977-2020) and quarter 3 (consistent 1991-2019)
    df1 = df1.loc[(df1['Quarter'] == 1) | (df1['Quarter'] == 3)]
   
    #set length class to cm and calculate weight
    df1["LngtClass_cm"]=df1["LngtClass"]/10
    
    #length to weight conversion
    df1["Weight"]=length_to_weight[i][0]*(df1["LngtClass_cm"]**length_to_weight[i][1])
    
    #Total weight in catch by multiplying by number caught per hour
    df1["Total_wght"]=df1["CPUE_number_per_hour"]*df1["Weight"]
    
    #Convert from grams to kilograms
    df1["Biomass_kg"]=df1["Total_wght"]/1000

    #Separate dataframe per length threshold between juvenile and adult class
        #for juvenile
    df1_juv=df1[df1['LngtClass_cm']< length_class_list[i]]
    df1_juv[species+"_"+life_stage[0]]=df1_juv["Biomass_kg"]
    df1_juv=df1_juv.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,-1]]
        #for adult
    df1_ad= df1[df1['LngtClass_cm']>length_class_list[i]]
    df1_ad[species+"_"+life_stage[1]]=df1_ad["Biomass_kg"]
    df1_ad=df1_ad.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,-1]]

    #concatenate back together with total juvenile and adult biomass caught per as separate column in same dataframe
    frames=[df1_juv,df1_ad,]
    occ_df = pd.concat(frames)
    occ_df2=occ_df.groupby(['Survey','Year','Quarter','DateTime','ShootLat','ShootLong','Depth'])[species+"_"+life_stage[0],
                                                                        species+"_"+life_stage[1]].sum().reset_index()
    
    #set datetime column values to datetime object
    occ_df2['DateTime'] = pd.to_datetime(occ_df2["DateTime"],dayfirst=True)
    
    #ensure depth column values always expressed in absolute value
    occ_df2['Depth'] = occ_df2['Depth'].abs()

    #Export consistent measured data 1977-2020 of Quarter 1
    occ_df3 = occ_df2.loc[(occ_df2['Quarter']==1)]

    #Sort by Datetime
    occ_df3= occ_df3.sort_values(['DateTime'], ascending=[True])
    occ_df3['DateTime']= occ_df3["DateTime"].dt.strftime("%d-%m-%Y %r")

    #Create new variable of unique Year/Quarter combination starting from 1 (default is zero)
    occ_df3['YearQuarter'] = make_identifier(occ_df3[['Year','Quarter']])
    occ_df3['YearQuarter'] = occ_df3['YearQuarter'] + 1
    
    occ_df3 = occ_df3.to_csv(file_dir+'{}_1977_2019_juv_adult_biomass.csv'.format(species))
    print(len(occ_df2))
    
    #Export consistent measured data 1991-2019 of Quarter 1 and 3
    occ_df3 = occ_df2.loc[(occ_df2['Year'] >= 1991) & (occ_df2['Year'] < 2020)]
    #Sort by Datetime
    occ_df3=occ_df3.sort_values(['DateTime'], ascending=[True])
    occ_df3['DateTime']= occ_df3["DateTime"].dt.strftime("%d-%m-%Y %r")
    #Create new variabe of unique Year/Quarter combination starting from 1 (default is zero)
    occ_df3['YearQuarter'] = make_identifier(occ_df3[['Year','Quarter']])
    occ_df3['YearQuarter'] = occ_df3['YearQuarter'] + 1
    
    occ_df3.to_csv(file_dir+'{}_1991_2019_juv_adult_biomass.csv'.format(species))
    print(len(occ_df3))

## Alternative: Length, Weight, Abundance (CPUE) - no age class

In [2]:
def make_identifier(df):
    str_id = df.apply(lambda x: '_'.join(map(str, x)), axis=1)
    return pd.factorize(str_id)[0]

#List of species included in research
species_list = ["Limanda_limanda","Merlangius_merlangus","Clupea_harengus","Gadus_morhua","Sprattus_sprattus",
                "Pleuronectes_platessa","Eutrigla_gurnardus","Amblyraja_radiata","Callionymus_lyra"]

# maturation size (based on FishBase)
length_class_list = [(11),  # Limanda limanda
                     (20.2),# Merlangius merlangus
                     (12.5),# Clupea harengus
                     (31),  # Gadus morhua
                     (8),   # Sprattus sprattus
                     (24),  # Pleuronectes platessa
                     (14.5),# Eutrigla gurnardus
                     (40),  # Amblyraja radiata
                     (17.4)]# Callionymus lyra


length_to_weight = [(0.0068,3.14), #parameter a,b  a*length^b conversion for Limanda limanda
                   (0.0063,3.06),  # ... Merlangius merlangus
                   (0.0060,3.08),  # ... Clupea harengus
                   (0.0071,3.08),  # ... Gadus morhua
                   (0.0056,3.09),  # ... Sprattus sprattus
                   (0.0089,3.04),  # ... Pleuronectes platessa
                   (0.0079,3.02),  # ... Eutrigla gurnardus
                   (0.0105,2.94),  # ... Amblyraja radiata
                   (0.0214,2.59)]  # ... Callionymus lyra

#North Sea coordinate extent:
min_lat= 48.00
max_lat= 62.00
min_lon= -5.00
max_lon= 13.01

#process per species
for i in range(len(species_list)):  
    species = species_list[i]
    print(species)
    maturation_size = length_class_list[i]
    print('maturation size '+ str(maturation_size) +" cm")
    df1=pd.read_csv(file_dir+species_list[i]+'.csv')

    #drop observations outside geographic range
    df1=df1.drop(df1[(df1["ShootLong"] < min_lon) & (df1["ShootLong"] > max_lon)].index)
    df1=df1.drop(df1[(df1["ShootLat"] < min_lat) & (df1["ShootLat"] > max_lat)].index)
    
    #Keep observations of quarter 1 (consistent 1977-2020) and quarter 3 (consistent 1991-2019)
    df1 = df1.loc[(df1['Quarter'] == 1) | (df1['Quarter'] == 3)]
   
    #set length class to cm and calculate weight
    df1["LngtClass_cm"]=df1["LngtClass"]/10
    
    #length to weight conversion
    df1["Weight"]=length_to_weight[i][0]*(df1["LngtClass_cm"]**length_to_weight[i][1])
    
    #Total weight in catch by multiplying by number caught per hour
    df1["Total_wght"]=df1["CPUE_number_per_hour"]*df1["Weight"]
    
    #Convert from grams to kilograms
    df1["Biomass_kg"]=df1["Total_wght"]/1000
    
    #Set maturity
    df1['Maturity']=0
    df1.loc[df1.LngtClass_cm >= maturation_size, ['Maturity']] = 1

    #set datetime column values to datetime object
    df1['DateTime'] = pd.to_datetime(df1["DateTime"],dayfirst=True)
    print(len(df1))
    #Export consistent measured data 1977-2020 of Quarter 1
    df2 = df1.loc[(df1['Quarter']==1)]
    print(len(df2))
    
    #Sort by Datetime
    df2 = df2.sort_values(['DateTime'], ascending=[True])
    df2['DateTime']= df2["DateTime"].dt.strftime("%d-%m-%Y %r")
    print(len(df2))
    #Create new variable of unique Year/Quarter combination starting from 1 (default is zero)
    df2['YearQuarter'] = make_identifier(df2[['Year','Quarter']])
    df2['YearQuarter'] = df2['YearQuarter'] + 1
    df2 = df2[['Survey','Year','Quarter','YearQuarter','DateTime','ShootLat','ShootLong','LngtClass_cm','CPUE_number_per_hour','Biomass_kg','Maturity']]
    df2.to_csv(file_dir+'{}_1977_2019_length_weight_cpue.csv'.format(species))
    print(len(df2))
    
    #Export consistent measured data 1991-2019 of Quarter 1 and 3
    df3 = df1.loc[(df1['Year'] >= 1991) & (df1['Year'] < 2020)]
    print(len(df3))
    #Sort by Datetime
    df3=df3.sort_values(['DateTime'], ascending=[True])
    df3['DateTime']= df3["DateTime"].dt.strftime("%d-%m-%Y %r")
    #Create new variabe of unique Year/Quarter combination starting from 1 (default is zero)
    df3['YearQuarter'] = make_identifier(df3[['Year','Quarter']])
    df3['YearQuarter'] = df3['YearQuarter'] + 1
    
    df3 = df3[['Survey','Year','Quarter','YearQuarter','DateTime','ShootLat','ShootLong','LngtClass_cm','CPUE_number_per_hour','Biomass_kg','Maturity']]
    
    df3.to_csv(file_dir+'{}_1991_2019_length_weight_cpue.csv'.format(species))
    print(len(df3))

Limanda_limanda
maturation size 11 cm
268295
176522
176522
176522
217988
217988
Merlangius_merlangus
maturation size 20.2 cm
350718
230122
230122
230122
272389
272389
Clupea_harengus
maturation size 12.5 cm
290556
198960
198960
198960
224351
224351
Gadus_morhua
maturation size 31 cm
166435
114115
114115
114115
117689
117689
Sprattus_sprattus
maturation size 8 cm
135798
105713
105713
105713
101382
101382
Pleuronectes_platessa
maturation size 24 cm
163483
102523
102523
102523
135268
135268
Eutrigla_gurnardus
maturation size 14.5 cm
170273
106100
106100
106100
151451
151451
Amblyraja_radiata
maturation size 40 cm
45032
30556
30556
30556
35839
35839
Callionymus_lyra
maturation size 17.4 cm
40845
24700
24700
24700
33960
33960
